In [1]:
import os
from openai import AzureOpenAI

api_key = os.getenv('AZURE_OPENAI_KEY')
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
version = os.getenv('AZURE_OPENAI_VERSION')
model = os.getenv('GPT_DEPLOYMENT_NAME')

client = AzureOpenAI(
                azure_endpoint=endpoint,
                api_key=api_key,
                api_version=version
            )

In [16]:
functions = [
    {
        "type":"function",
        "function" :{
            "name": "summarize_text",
            "description" : "summarize text into sections",
            "strict": True,
            "parameters":{
                "type": "object",
                "properties":{
                    "summary":{
                        "type": "string",
                        "description": "Summary of the text divided into sections"
                    }
                },
                "required":["summary"],
                "additionalProperties": False
            }    
        }
    }
    ]

In [2]:
def get_completion(messages: list, temperature:float = 0.7) -> str:
    
        completion = client.chat.completions.create(
            model = model, 
            messages=messages,
            temperature=temperature,
            #tools=functions,
            #tool_choice={"type": "function", "function": {"name": "summarize_text"}}
        )

        return completion.choices[0].message.content#tool_calls[0].function.arguments

## agents solution

1. Topic choice
2. Function to identify what kind of outcome is the user expecting (pptx, pdf, image with text (display))
3. Receive as input one of the previous categories (category = "pdf")
4. Decide which prompts to use based on output of previous function.


In [3]:
summarization_text_prompt = '''You're going to receive two inputs, a context and a user query.
Please read and understand the context below delimited by triple backticks and the user query and follow the next instructions.

### CONTEXT ###
```{context}```

### QUERY ###
```{query}```

Follow this instructions:
### INSTRUCTIONS ###
1. Read carefully and understand the context
2. Create a summary of the context. The summary must have the most important elements on the text.
3. Make sure that the summary contains the necessary information that the user is looking for in the query.
4. Split your summary into sections and short paragraphs.

Return a string with a valid json format. Use the sections as key names. Do not include the word 'json' as part of your respond.
'''

In [4]:
text = """
"[69] \n \nREFERENCES \n \nA., A. (2023). The Renaissance of Creativity: Exploring the Impact of Generative AI in the Creative \nIndustry. https://www.linkedin.com/pulse/renaissance -creativity-exploring-impact-generative-ai-\nadam \nAela, E. (2023, April 1). Artificial Intelligence: How AI is Changing Art. \nhttps://aelaschool.com/en/art/artificial-intelligence-art-changes/ \nAhmed, R. (2023). The Future of Art and Creativity in the Age of AI. \nhttps://www.linkedin.com/pulse/future-art-creativity-age-ai-rohan-ahmed \nGwafan, D. (2023). Python Programming Language: A Versatile and Powerful Tool for Modern \nDevelopment. https://www.linkedin.com/pulse/python -programming-language-versatile-powerful-\ntool-modern-gwafan \nGenerative art. (2023). In Wikipedia. \nhttps://en.wikipedia.org/w/index.php?title=Generative_art&oldid=1176498478 \nHarwell, D. (2022). He used AI art from Midjourney to win a fine -arts prize. Did he cheat? - The \nWashington Post. https://www.washingtonpost.com/technology/2022/09/02/midjourney -artificial-\nintelligence-state-fair-colorado/ \nMahadik, S. (2023). Transformation of Creative Industries with Generative AI. \nhttps://www.linkedin.com/pulse/transformation-creative-industries-generative-ai-sandesh-mahadik \nMatplotlib. (2023). In Wikipedia. \nhttps://en.wikipedia.org/w/index.php?title=Matplotlib&oldid=1177283978 \nNigeria,  consistent_hippopotamus | R. C. A. |. (2023). Artists are not happy with the existence of AI. \nTopical Talk. https://talk.economistfoundation.org/festivals/festival-2023/ai-and-the-arts/artists-are-\nnot-happy-with-the-existence-of-ai/ \nNumPy. (2023). In Wikipedia. https://en.wikipedia.org/w/index.php?title=NumPy&oldid=1174952220 \nPearson, M. (2011). Generative art, a practical guide using processing. \nPip (package manager). (2023). In Wikipedia. \nhttps://en.wikipedia.org/w/index.php?title=Pip_(package_manager)&oldid=1161551744 \nPyCharm. (2023). In Wikipedia. \nhttps://en.wikipedia.org/w/index.php?title=PyCharm&oldid=1172673606 \nPython, R. (2023). The Beginner’s Guide to Python Turtle – Real Python. \nhttps://realpython.com/beginners-guide-python-turtle/ \nTempel, M. (2017). Generative art for all. Journal of Innovation and Entrepreneurship , 6(1). \nhttps://www.academia.edu/88354378/Generative_art_for_all\n\n[59] \n \nproduce more advanced and higher-quality work and has helped visual artists produce art. AI \nis modernizing some processes, but it still requires human involvement to create something \nuseful and, most importantly, interesting. Therefore, AI can be seen as a tool that complements \nand enhances the work of designers and artists to make their work more productive and efficient. \nDesigners can fill the gap between human touch and creativity, which is fundamental to the \nartwork. Unlike computers, artists are endowe d with their own emotions and unique \nperspectives (Nigeria, 2023). \nGenerative art, also known as procedural art or algorithmic art, is a form of art that \ncreates itself independently. This can take the form of mathematical functions that create \ngeometric and precisely calculated works of art. In other cases, chance takes control and creates \nchaotic artworks that incorporate abstract styles. In many cases, digital artists combine the two. \nGenerative art comes in a variety of forms, from music to literature to digital graphics. By \ndefinition, generative art is a form of art that is either entirely or partially created using an \nautonomous system. Generative art is a unique combination of computer science and art. This \nsystem can be represented by any type of algorithm: mathematical, mechanical, or biological. \nGenerative art creation usually involves a programming language with the use of codes. \nSimilarly, generative content that uses AI uses prompt technology to generate the desired \ninformation. Investigating the Python programming language for generating artwo rk will \ncontribute to the understanding of generative art and provide important and meaningful insights \ninto artificial intelligence. \nThis type of art is often inspired by modern art, especially Pop Art, which relies heavily \non ordered geometric patterns. However, it is a very broad and rich category of art that has one \ncentral characteristic. Generative art in some way involves a self -governing or autonomous \nsystem. \nChance is a kind of autonomous system. By incorporating chance into a piece of code \nart, different unique pieces of art are generated each time the script is executed or in response \nto the user's interaction. This can create different types of ideas that are available for selection. \nThis randomness can also be orderly and autonomous or the integration of mixing and chaos. \nAutonomous or generative art is not rejecting creativity, it finds creativity through \ndifferent fields by building art pieces through coding. Artists may argue that generative arts will \nkill manual art eventually. A game designer from Colorado USA recently took home the first \nprize in the digital arts or digitally manipulated photography category at the Colorado State Fair \nFine Arts Competition. His winning image was made with Midjourney - an Artificial \nIntelligence system that can come up with images at a prompt. The win was met with resistance \nas many would argue that using AI would be as easy as typing prompts. However, to come up \nwith such complicated imagery, there were a lot of things involved. It took more than 80 hours. \nThis example of generative art does not involve any coding but uses prompts, tweaks, light \nadjustments, and Photoshop to come up with the final art piece which initially needed 900 \niterations apart from using more AI programs to improve resolution (Harwell, 2022). \n \nRESEARCH QUESTIONS \n1. What is generative art, and how can Python programming language be utilized \nin its creation? \nGenerative art is a form of digital art that uses algorithms, often aided by computers and \nsoftware, to create artwork. Rather than creating art manually through traditional techniques \nsuch as painting or sculpting, generative artists write code that defin es rules, processes, and\n\n[58] \n \n \n \nUNDERSTANDING OF GENERATIVE ART WITH ‘PYTHON’ \nPROGRAMMING \n \n \nTeddy Marius Soikun1* \n \n1Academy of Arts and Creative Technology,  \nUniversity Malaysia Sabah, 88400 Kota Kinabalu, Malaysia  \n*tsoikun@ums.edu.my \n \nReceived: 21 June 2023 | Revised: 10 July 2023  \nAccepted: 30 September 2023 / Published: 20 December 2023 \nDOI: https://10.51200/ga.v13i2.4735     \n \n \nABSTRACT \nThe article uses the Python programming language to introduce generative arts as an alternative \nto conventional art and design. Understanding generative arts in the age of Artificial \nIntelligence (AI) is seen as a way forward to remain relevant. Through the use of a practice -\nbased research methodology, the article explores principles, processes, and tools for generating \nart through coding. Practical samples are used to demonstrate the potential of generative art \nthrough basic coding examples. This research also highlights the role of generative art in \nfostering interdisciplinary collaboration and transcending the boundaries of conventional art \nforms. \nKeywords: Generative Arts, Python, Artificial Intelligence \n \nINTRODUCTION \nThe computer plays a very important role in our life nowadays. Almost every task is connected \nto the computer. From ordering groceries online to using public transportation to driving a car \nor communicating with other people. One part that plays an even mo re important role in \ntechnology is artificial intelligence. Although it has been with us for quite some time, the term \nstill inspires fear and awe. Most of us are still wondering about its potential and future. AI can \nbe used in managing traffic routes, pr oducts, movies, and location suggestions, and as virtual \nassistants such as chatbots, search engines, banks, and so on (Aela, 2023). \nRecently, the art world has been transformed by the advent of technology, especially \nAI and generative arts. Many types of art can now be easily made with the help of a computer. \nWith the advancement of AI, many high -quality artworks can be created with less time and \nresources. The technology has achieved significant results in generating visual, animated, and \naudio content (Hodgkins, 2023). \nAI is more or less comparable to the advent of photography more than a hundred years \nago. Everyone was shocked when it first appeared. However, it has become a tool for artists to\n\n[65] \n \nsimple generative art piece may give insights into how the technology works and may \nlead to the understanding of AI generative content in art.  \n Explanation  Sample codes \n1 Python must be installed in the \ncomputer as well as the \nrequired libraries. matplotlib is \ninstalled using pip: \n \n2 Following PBR, the researcher \ndefine the generative art piece. \nThe art piece should be pixel \nbased coloured concept. This \nis designed for a digital poster \nbackground.  \n \n3 To use the Python’s library, it \nmust be installed in the \ncomputer. It is then imported \nin the current project. The \nresearcher uses matplotlib for \nplotting and numpy for \nnumerical operations. \n \n4 Because it is an artpiece, \nsimilarly to some design \nsoftware, a blank canvas \nshould be created. NumPy is \nused with the specified width \nand height. \n \n5 The art is generated by \nmanipulating the canvas. \n \nThe parameters are defined by \nthe spiral and iterate through a \nrange of points. For each \npoint, the position is \ncalculated using polar \ncoordinates, and a random \ncolor is assigned to it. \n \n \n6 Matplotlib is used to display \nthe generated art. The axis are \nturned off to make it purely \nvisual."
"""

In [5]:
user_prompt = summarization_text_prompt.format(context = text, query = "generative ai")

messages = [{"role":"user", "content":user_prompt}]

summary = get_completion(messages)

In [58]:
instruct = """
Describe in detail the process of changing my car's engine oil.
"""

messages = [{"role": "user", "content": instruct}]

result = get_completion(messages)

print(result)

Changing your car's engine oil is a crucial maintenance task that helps ensure the longevity and optimal performance of your vehicle. Below is a detailed step-by-step guide to help you through the process:

### Tools and Materials Needed:

1. **Tools:**
   - Socket wrench or ratchet set
   - Oil filter wrench
   - Funnel
   - Oil drain pan
   - Rags or paper towels
   - Jack and jack stands (or ramps)

2. **Materials:**
   - New engine oil (check your owner's manual for the correct type and amount)
   - New oil filter
   - Oil filter gasket (if needed)
   - Oil drain pan (if not using one already)
   - Gloves (optional)
   - Safety glasses (optional)

### Step-by-Step Process:

#### 1. Preparation:
   - **Check the Owner's Manual:** Before you start, refer to your vehicle's owner manual for specific oil change recommendations, including the type of oil and filter required.
   - **Gather Materials:** Make sure you have all necessary tools and materials on hand.

#### 2. Safety First:
  

In [60]:
prompt2 = f"""
Below there's a text that describes the process of how to change the car's engine oil.

Understand the text and create a short sentece that sums up the text.

Create your summary as if you were describing a process and then create a draw of it.
Be as specific as you can, the text you generate is going to be used as prompt to a dalle-3 model to generate images.

###TEXT###
{result}

return only the drawing prompt for dall-e 3
"""

messages = [{"role": "user", "content": prompt2}]

description = get_completion(messages)

print(description)

"An illustration of the step-by-step process of changing a car's engine oil: 1) Tools and materials laid out on a garage floor, including a wrench set, oil filter wrench, funnel, oil drain pan, and new engine oil. 2) A car parked on a level surface with the parking brake engaged. 3) The car lifted with a jack and secured with jack stands. 4) A person draining old oil from the oil pan into a drain pan, wearing gloves. 5) Removing the old oil filter with an oil filter wrench. 6) Installing a new oil filter, applying oil to the gasket. 7) Replacing the drain plug after draining. 8) Pouring new engine oil into the engine using a funnel. 9) Checking the oil level with a dipstick. 10) Properly disposing of old oil at a recycling center. Show a clean, organized garage environment with bright lighting."


In [61]:
description

'"An illustration of the step-by-step process of changing a car\'s engine oil: 1) Tools and materials laid out on a garage floor, including a wrench set, oil filter wrench, funnel, oil drain pan, and new engine oil. 2) A car parked on a level surface with the parking brake engaged. 3) The car lifted with a jack and secured with jack stands. 4) A person draining old oil from the oil pan into a drain pan, wearing gloves. 5) Removing the old oil filter with an oil filter wrench. 6) Installing a new oil filter, applying oil to the gasket. 7) Replacing the drain plug after draining. 8) Pouring new engine oil into the engine using a funnel. 9) Checking the oil level with a dipstick. 10) Properly disposing of old oil at a recycling center. Show a clean, organized garage environment with bright lighting."'

In [40]:
clean_sum = summary.replace("`", "").replace("json","")

In [7]:
import json

sum_json = json.loads(summary)

In [8]:
sum_json

{'Definition': 'Generative AI refers to a form of art, known as generative art, which is created using algorithms and autonomous systems. It combines elements of computer science and art, allowing artists to produce works through coding instead of traditional manual techniques.',
 'Role of AI in Art': 'AI is transforming the creative industries by enabling artists to produce high-quality artworks more efficiently. While AI serves as a powerful tool for enhancing artistic productivity, it still requires human involvement to infuse creativity and emotional depth into the artwork.',
 'Generative Art Characteristics': 'Generative art can take various forms, including visual art, music, and literature. It often involves randomness and can produce unique pieces each time a script is executed. The art is generated through coding, utilizing programming languages like Python, which allows for manipulation of digital canvases and incorporation of complex algorithms.',
 'Python and Generative Art

In [12]:
import json

json_output = """
"text" : "{
    "abstract": "The article discusses the use of Python programming to explore generative art as a modern approach to art and design influenced by Artificial Intelligence (AI). It emphasizes the relevance of understanding generative art and its processes, highlighting the potential for interdisciplinary collaboration.",
    "introduction": "The rise of technology, particularly AI, has transformed the art world, allowing for the creation of high-quality artworks efficiently. Generative art, which is created through algorithms and autonomous systems, merges art and computer science, enabling both randomness and structured creativity.",
    "generative_art": "Generative art involves the creation of artworks through coded algorithms, contrasting with traditional art forms. It can include various art types and often relies on the use of programming languages like Python, which offers libraries such as Pygame and Processing.py for artists to create visual and interactive pieces.",
    "python_usage": "Python is favored for generative art due to its simplicity, extensive libraries, and readability. Artists can generate artworks through algorithms that define rules and parameters. Examples include fractal patterns, particle systems, and neural networks, each providing unique visual outputs based on coded instructions.",
    "conclusion": "By utilizing Python, artists can leverage technology to enhance creativity and produce innovative art forms. Generative art exemplifies the interplay between human creativity and machine capabilities, fostering new artistic expressions through coding."
}"
"""

#res = json.loads(json_output)

In [13]:
description_prompt = f"""You will receive an input text in json format.
Your task is to identify the keys and return them as a list.

### INPUT TEXT ###
{json_output}

return only the list.
"""

messages = [{"role":"user", "content":description_prompt}]

keys = get_completion(messages)
print(keys)

```json
["abstract", "introduction", "generative_art", "python_usage", "conclusion"]
```


In [29]:
jj = """
```json\n{\n  \"overview\": \"The article highlights how Python programming serves as a modern alternative to traditional art forms through the creation of generative art, focusing on the principles, processes, and tools essential for art generation using coding and artificial intelligence.\",\n  \"important_concepts\": \"Generative art is defined as artworks produced by autonomous systems or algorithms, primarily using programming languages like Python, emphasizing the importance of algorithms in establishing creative rules, the role of randomness in art, and the synergy between human creativity and computational power.\",\n  \"features\": \"Python is preferred for generating art due to its user-friendly nature and a wide range of libraries, including Pygame, Processing.py, and Turtle Graphics, enabling artists to create stunning visuals, animations, and interactive experiences, such as fractal patterns, particle system simulations, and realistic image generation through neural networks.\",\n  \"relevance\": \"Generative art challenges conventional ideas about authorship and creativity, encouraging a fresh perspective on the interplay between human and machine-generated art, fostering collaboration between artists and programmers, and demonstrating the transformative impact of technology on artistic expression.\",\n  \"conclusion\": \"Utilizing Python for generative art empowers artists to innovate and express themselves by exploiting algorithms and computational systems, allowing for the creation of distinctive and intricate artworks that fuse creativity with cutting-edge technology.\"\n}\n```
"""

kk = jj.replace("`", "").replace("json","").replace(","," ")

In [57]:
from datetime import datetime

images_folder = "../api/assets/generated_images"

sub_folder = (datetime.today().strftime('%Y-%m-%d %H:%M:%S')).replace(" ","-").replace(":","-")
images_path = os.path.join(images_folder,sub_folder)

if not os.path.exists(images_path):
   os.makedirs(images_path) 

images_path

'../api/assets/generated_images\\2025-02-27-14-57-00'

## pictures test

In [3]:
from openai import AzureOpenAI

openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
dalle_model = os.getenv('GPT_IMAGES_DEPLOYMENT_NAME')
gpt_model = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')

llm = AzureOpenAI(
    api_key=openai_api_key,
    #model=gpt_model,
    azure_endpoint = openai_api_base,
    api_version=openai_api_version
)

In [2]:


import os

openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
dalle_model = os.getenv('GPT_IMAGES_DEPLOYMENT_NAME')
gpt_model = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')


In [4]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
from langchain.chains import LLMChain
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_core.prompts import PromptTemplate
import os
from langchain_openai import AzureOpenAIEmbeddings, AzureOpenAI
#from openai import AzureOpenAI

openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
embeddings_model = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
gpt_model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
dalle_model = os.getenv("GPT_IMAGES_DEPLOYMENT_NAME")

dalle_client = AzureOpenAI(
          model = dalle_model,
          api_key=openai_api_key,
          api_version=openai_api_version,
          azure_endpoit=openai_api_base
    )

def langchain_imgs(prompt: str):
        """Generate images based on user queries"""

        prompt_ = PromptTemplate(
            input_variables=["image_desc"],
            template="Generate a detailed prompt to generate an image based on the following query: {image_desc}",
        )

        chain = LLMChain(llm=dalle_client, prompt=prompt_)

        img_url = DallEAPIWrapper().run(chain.run(prompt))

        return img_url

ValidationError: 1 validation error for AzureOpenAI
  Value error, As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). [type=value_error, input_value={'model': 'dall-e-3', 'ap...m/', 'model_kwargs': {}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [17]:
dalle_model

'dall-e-3'

In [15]:
dalle_client = AzureOpenAI(
          model = dalle_model,
          api_key=openai_api_key,
          api_version=openai_api_version,
          azure_endpoint=openai_api_base
    )

In [16]:
dalle_client.invoke("hello")

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}